In [ ]:

def print_board(board, li=0, lj=0):
    """Print the game board.

    The arguments 'li' and 'lj' are used to provide
    the coordinates of the last stone that was placed on the board.
    """

    print("\033c\n")
    print("    \033[31m0 1 2 3 4 5 6\033[0m")
    if board[li, lj] == 2:
        # It is set to 3 to denote that it's a new blue stone.
        # It'll be fixed after printing.
        board[li, lj] = 3

    for i in range(7):
        print(f"{' '*i} \033[34m{i}\033[0m  ", end="")

        for j in range(7):
            if board[i, j] == 1:      # Red
                print("\033[31m●\033[0m ", end="")
            elif board[i, j] == 2:      # Blue
                print("\033[34m●\033[0m ", end="")
            elif board[i, j] == 3:      # newly placed Blue
                print("\033[34m◙\033[0m ", end="")
                board[i, j] = 2     # not new anymore
            else:
                print("- ", end="")
        print(f" \033[34m{i}\033[0m ")

    print(" "*10, "\033[31m0 1 2 3 4 5 6\033[0m")
    print("_"*30, "\n")


def get_neighbors(board, i, j):
    """Return a list of same colored neighbors of a stone in [i, j]."""
    neighbors = []
    cells = [(i, j-1), (i, j+1), (i-1, j), (i+1, j), (i-1, j+1), (i+1, j-1)]
    # For all six cells around this one:
    for ni, nj in cells:
        # If it's not out of the borders and has a stone with the same color:
        if 0 <= ni < 7 and 0 <= nj < 7 and board[ni, nj] == board[i, j]:
            neighbors.append((ni, nj))
    return neighbors


def check_win(board, connections, i, j):
    """Check if the game is over after placing a stone in [i, j]."""
    # The side that this stone is connected to:
    side = connections[i, j]
    for ni, nj in get_neighbors(board, i, j):
        # If the stone has a connected neighbor:
        if connections[ni, nj]:
            # If the stone is also connected to the other side:
            if side and side != connections[ni, nj]:
                return board[i, j]      # return the winner
            side = connections[ni, nj]
    return 0


def update_connectivity(board, connections, i, j):
    """Update the connectivity status of this stone."""
    neighbors = get_neighbors(board, i, j)

    # Connected through its neighbors:
    for ni, nj in neighbors:
        if connections[ni, nj]:
            connections[i, j] = connections[ni, nj]
            break

    # Directly connected to a side:
    if board[i, j] == 1:
        # A red stone could be connected to the top or bottom row.
        if i == 0:
            connections[i, j] = 1
        elif i == 6:
            connections[i, j] = 2
    elif board[i, j] == 2:
        # A blue stone could be connected to the left or right column.
        if j == 0:
            connections[i, j] = 1
        elif j == 6:
            connections[i, j] = 2

    # If this stone is updated, also update all of its neighbors.
    if connections[i, j]:
        for ni, nj in neighbors:
            # A neighbor that already has a value doesn't need to be updated.
            if not connections[ni, nj]:
                update_connectivity(board, connections, ni, nj)


def place_stone(board, connections, player, i, j):
    """Place a stone in coordinate [i, j] of the board.

    Returns whether the stone was placed successfully or not.
    """

    # If the input is invalid (out of borders or already filled):
    if i < 0 or i > 6 or j < 0 or j > 6 or board[i, j] != 0:
        return False
    board[i, j] = player
    update_connectivity(board, connections, i, j)
    return True


In [ ]:

import numpy as np


def get_group(board, i, j, group, size):
    """Given a stone in coordinates [i, j], find all stones
    that are connected to it; constituting a group.

    Size of this group specifies the start and end
    of its width or height, depending on the color.
    """

    # Either width or height. The other one doesn't matter.
    s = i if board[i, j] == 1 else j
    # Find minimum
    if s < size[0]:
        size[0] = s
    # Find maximum
    if s > size[1]:
        size[1] = s

    group.append((i, j))
    for ni, nj in get_neighbors(board, i, j):
        if (ni, nj) not in group:
            get_group(board, ni, nj, group, size)


def heuristic_for_color(board, player):
    """Given a board, calculate the heuristic value for only one color.

    Heuristic value is calculated based on the lengths of groups for a color.
    It is the result of concatenating the lengths
    of top four longest groups (if exist).

    For example if the color blue has three groups with the following length:
    [1, 3, 1]
    The heuristic value will be: 3110
    """

    seen = set()
    lengths = []
    for i in range(7):
        for j in range(7):
            # If this stone is not in one of the groups that have been seen:
            if board[i, j] == player and (i, j) not in seen:

                # Result of 'get_group' will be saved in these arrays:
                group = []
                size = [7, -1]
                get_group(board, i, j, group, size)
                lengths.append(str(size[1] - size[0] + 1))
                seen = seen.union(group)

    # Calculate the heuristic value
    lengths.sort(reverse=True)
    return int("".join(lengths)[:4].ljust(4, "0"))


# def heuristic_for_color(board, player):
#     """Heuristic function based on the length of longest group."""
#     seen = set()
#     max_length = 0
#
#     for i in range(7):
#         for j in range(7):
#             if board[i, j] == player and (i, j) not in seen:
#                 group = []
#                 size = [7, -1]
#                 get_group(board, i, j, group, size)
#
#                 length = size[1] - size[0] + 1
#                 if length > max_length:
#                     max_length = length
#                 seen = seen.union(group)
#
#     return max_length


def negamax(board, connections, player, alpha, beta, depth):
    """The Negamax algorithm with alpha-beta pruning.

    Note: The heuristic value is equal to the difference of calling
    the 'heuristic_for_color' function for both colors.
    """

    best_value = -np.inf
    best_move = ()
    if depth <= 0:
        # Heuristic value for the opponent
        opp_value = heuristic_for_color(board, player^3)

    for i in range(7):
        for j in range(7):
            if board[i, j] == 0:
                # Trial move with stone in [i, j]
                board_c = board.copy()
                connections_c = connections.copy()
                place_stone(board_c, connections_c, player, i, j)

                if check_win(board_c, connections_c, i, j):
                    return 20000, (i, j)
                if depth <= 0:
                    # The heuristic value
                    value = heuristic_for_color(board_c, player) - opp_value
                else:
                    value = -negamax(board_c, connections_c, player^3,
                                     -beta, -alpha, depth-1)[0]

                if value > best_value:
                    best_value = value
                    best_move = (i, j)
                alpha = max(alpha, value)
                if beta <= alpha:
                    # Prune
                    return best_value, best_move

    return best_value, best_move


def agent(board, connections):
    """The AI agent.

    Given a board, it will choose a move by
    running the Negamax algorithm with the specified depth.
    """

    return negamax(board, connections, 2, -10000, 10000, 3)[1]


In [ ]:

def agent(board, connections):
    """The AI agent.

    Given a board, it will choose a move by
    running the Negamax algorithm with the specified depth.
    """

    return negamax(board, connections, 2, -10000, 10000, 3)[1]


In [ ]:

# The game board.
# 0 means empty, 1 means red and 2 means blue.
board = np.zeros((7, 7), dtype="int8")

# Whether or not a stone is connected to a side.
# 0 means it's not connected to either of the sides of its color.
# 1 means it's connected to the first side and
# 2 means it's connected to the other side.
connections = np.zeros((7, 7), dtype="int8")

## For test
# board = np.array([
# [0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 0, 0, 0, 0],
# ])
# connections = np.array([
# [0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 0, 0, 0, 0],
# ])


if __name__ == "__main__":

    # 1 is the player (red) and 2 is the agent (blue)
    player = 1 if input("\nDo you want to go first? ").lower() != "n" else 2
    print_board(board)
    game_over = False

    while not game_over:
        if player == 1:
            placed = False
            while not placed:
                inp = input("Enter stone: ").split()
                i, j = int(inp[0]), int(inp[1])
                placed = place_stone(board, connections, player, i, j)
        elif player == 2:
            i, j = agent(board, connections)
            place_stone(board, connections, player, i, j)

        print_board(board, i, j)
        game_over = check_win(board, connections, i, j)
        player ^= 3     # Switch players

    winner = "\033[31;5mRED" if game_over == 1 else "\033[34;5mBLUE"
    print("Winner:", winner, "\033[0m")
